## 概要
二次微分まで可能な目的関数に対して、モーメンタムを追加した勾配法を用いることでサンプル効率が良くなりました。BERTの事前学習に使われているらしい。良い論文だけどimplicit gradient transportの論文を見たほうが、SGD改善のお気持ちが理解できるかも。

## 準備
* 最小化をするための目的関数
$$
F(\vec{w})=\underset{\xi \sim \mathcal{D}}{\mathbb{E}}[f(\vec{w}, \xi)]
$$
* SGDにおけるパラメータ更新
$$
\vec{w}_{t+1}=\vec{w}_t-\eta_t \nabla f\left(\vec{w}_t, \xi_t\right)
$$
## SGDを効率的に解く方法
パラメータ更新における学習率を適応的に変更させて効率的に最適化問題を解くということを考えます。AdaGradと言われる手法です。これに**モーメンタム**という概念を追加したのがAdamという有名なアルゴリズムです。モーメンタムを用いたパラメータ更新は以下のように書きます。
$$
\begin{aligned}
\vec{m}_t & =\beta \vec{m}_{t-1}+(1-\beta) \nabla f\left(\vec{w}_t, \xi_t\right) \\
\vec{w}_{t+1} & =\vec{w}_t-\eta_t \vec{m}_t
\end{aligned}
$$
モーメンタムは過去の勾配の移動平均を用いて計算されます。これがどうやら良い動きをしているらしいです。

その次に用いられる工夫としてモーメンタムを正規化したパラメータ更新があります。
$$
\begin{aligned}
\vec{m}_t & =\beta \vec{m}_{t-1}+(1-\beta) \nabla f\left(\vec{w}_t, \xi_t\right) \\
\vec{w}_{t+1} & =\vec{w}_t-\eta_t \frac{\vec{m}_t}{\left\|\vec{m}_t\right\|}
\end{aligned}
$$
直感としては、勾配の大きさを直接用いてパラメータを更新するより、勾配の方向を与えてパラメータ更新をしたほうが最適化をする際に有用な情報を与えるからと言われています。これらの"らしい"を理論的に深堀した論文がこの論文です。

## 定理1
$$
\frac{1}{T} \sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq \frac{29 \sqrt{R L}}{\sqrt{T}}+\frac{21 \sqrt{\sigma}(R L)^{1 / 4}}{T^{1 / 4}}+\frac{8 \sigma}{\sqrt{R L T}}
$$
### 証明
 $\hat{\epsilon}_t=\vec{m}_t-\nabla F\left(\vec{w}_t\right)$ とし、 $\epsilon_t=\nabla f\left(\vec{w}_t, \xi_t\right)-\nabla F\left(\vec{w}_t\right)$、$S(a, b)=\nabla F(a)-\nabla F(b)$とします。このとき、

$$
\begin{aligned}
\mathbb{E}\left[\left\|\epsilon_t\right\|^2\right] & \leq \sigma^2 \\
\mathbb{E}\left[\left\langle\epsilon_i, \epsilon_j\right\rangle\right] & =0 \text { for } i \neq j
\end{aligned}
$$
が成立します。任意の $t \geq 1$について、:

$$
\begin{aligned}
\vec{m}_{t+1} & =(1-\alpha)\left(\nabla F\left(\vec{w}_t\right)+\hat{\epsilon}_t\right)+\alpha \nabla f\left(\vec{w}_{t+1}, \xi_{t+1}\right) \\
& =\nabla F\left(\vec{w}_{t+1}\right)+(1-\alpha)\left(S\left(\vec{w}_t, \vec{w}_{t+1}\right)+\hat{\epsilon}_t\right)+\alpha \epsilon_{t+1} \\
\hat{\epsilon}_{t+1} & =(1-\alpha) S\left(\vec{w}_t, \vec{w}_{t+1}\right)+(1-\alpha) \hat{\epsilon}_t+\alpha \epsilon_{t+1}
\end{aligned}
$$
が成り立ちます。最初の定義を代入しただけですね。回帰的に解くと、

$$
\hat{\epsilon}_{t+1}=(1-\alpha)^t \hat{\epsilon}_1+\alpha \sum_{\tau=0}^{t-1}(1-\alpha)^\tau \epsilon_{t+1-\tau}+(1-\alpha) \sum_{\tau=0}^{t-1}(1-\alpha)^\tau S\left(\vec{w}_{t-\tau}, \vec{w}_{t+1-\tau}\right)
$$
ノルムでとります。三角不等式を用いて、
$$
\left\|\hat{\epsilon}_{t+1}\right\| \leq(1-\alpha)^t\left\|\epsilon_1\right\|+\alpha\left\|\sum_{\tau=0}^{t-1}(1-\alpha)^\tau \epsilon_{t+1-\tau}\right\|+(1-\alpha) \eta L \sum_{\tau=0}^{t-1}(1-\alpha)^\tau
$$
期待値をとります。右辺第二項は、 $\epsilon_t=\nabla f\left(\vec{w}_t, \xi_t\right)-\nabla F\left(\vec{w}_t\right)$と$\underset{\xi}{\mathbb{E}}\left[\|\nabla f(\vec{w}, \xi)-\nabla F(\vec{w})\|^2\right] \leq \sigma^2$を用いて変形しています。もちろん第3項は確率変数が含まれていないので期待値をとっても変わりません。
$$
\begin{aligned}
\mathbb{E}\left[\left\|\hat{\epsilon}_{t+1}\right\|\right] & \leq(1-\alpha)^t \mathbb{E}\left[\left\|\epsilon_1\right\|\right]+\alpha \sqrt{\sum_{\tau=0}^{t-1}(1-\alpha)^{2 \tau} \sigma^2}+(1-\alpha) \eta L \sum_{\tau=0}^{t-1}(1-\alpha)^\tau \\
& =(1-\alpha)^t \sigma+\frac{\alpha \sigma}{\sqrt{1-(1-\alpha)^2}}+\frac{\eta L}{\alpha} \\
& \leq(1-\alpha)^t \sigma+\sqrt{\alpha} \sigma+\frac{\eta L}{\alpha} \\
\mathbb{E}\left[\sum_{t=1}^T\left\|\hat{\epsilon}_t\right\|\right] & \leq \frac{\sigma}{\alpha}+T \sqrt{\alpha} \sigma+\frac{T \eta L}{\alpha}
\end{aligned}
$$
補題2($\sum_{t=1}^T\left\|\nabla F\left(\vec{w}_t\right)\right\| \leq \frac{3 F\left(\vec{w}_1\right)}{\eta}+\frac{3 L T \eta}{2}+8 \sum_{t=1}^T\left\|\hat{\epsilon}_t\right\|$)を用いて、
$$
\begin{aligned}
\sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] & \leq \frac{3 F\left(\vec{w}_1\right)}{\eta}+\frac{3 L T \eta}{2}+8 \sum_{t=1}^T\left\|\hat{\epsilon}_t\right\| \\
& \leq \frac{3 R}{\eta}+\frac{3 T L \eta}{2}+\frac{8 \sigma}{\alpha}+8 T \sqrt{\alpha} \sigma+\frac{8 T L \eta}{\alpha} \\
& \leq \frac{3 R}{\eta}+\frac{8 \sigma}{\alpha}+8 T \sqrt{\alpha} \sigma+\frac{10 T L \eta}{\alpha}
\end{aligned}
$$
あとは定数をいろいろいじると、定理の右辺と同じになります。


## 定理3
成立するための条件は論文参照
$$
\frac{1}{T} \sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq \frac{5 \sqrt{R L}}{\sqrt{T}}+\frac{8 \sigma^{13 / 7}}{R^{4 / 7} \rho^{2 / 7} T^{3 / 7}}+\frac{27 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7}}{T^{2 / 7}}
$$
## 証明
の前半は、定理1とほぼ同じ
$$
\begin{aligned}
\vec{m}_{t+1}= & (1-\alpha)\left(\nabla F\left(\vec{w}_t\right)+\hat{\epsilon}_t\right)+\alpha \nabla f\left(\vec{x}_{t+1}, \xi_{t+1}\right) \\
= & (1-\alpha)\left(\nabla F\left(\vec{w}_{t+1}\right)+\nabla^2 F\left(\vec{w}_{t+1}\right)\left(\vec{w}_t-\vec{w}_{t+1}\right)\right)+(1-\alpha)\left(Z\left(\vec{w}_t, \vec{w}_{t+1}\right)+\hat{\epsilon}_t\right)+\alpha \nabla F\left(\vec{w}_{t+1}\right) \\
& +\alpha\left(\frac{1-\alpha}{\alpha} \nabla^2 F\left(\vec{w}_{t+1}\right)\left(\vec{w}_{t+1}-\vec{w}_t\right)\right)+\alpha\left(Z\left(\vec{x}_{t+1}, \vec{w}_{t+1}\right)+\epsilon_{t+1}\right) \\
= & \nabla F\left(\vec{w}_{t+1}\right)+(1-\alpha) Z\left(\vec{w}_t, \vec{w}_{t+1}\right)+\alpha Z\left(\vec{x}_{t+1}, \vec{w}_{t+1}\right)+(1-\alpha) \hat{\epsilon}_t+\alpha \epsilon_{t+1} \\
\hat{\epsilon}_{t+1}= & (1-\alpha) \hat{\epsilon}_t+(1-\alpha) Z\left(\vec{w}_t, \vec{w}_{t+1}\right)+\alpha Z\left(\vec{x}_{t+1}, \vec{w}_{t+1}\right)+\alpha \epsilon_{t+1}
\end{aligned}
$$
定理3の証明における重要なポイントは、**Implicit gradient transportを用いて二次勾配の項を消せる**という部分です。ここが本質に思えるので、詳しく知りたい場合はこれを提案している論文を読みましょう。

$Z(a, b) \leq \rho\|a-b\|$ より、
$$
\hat{\epsilon}_{t+1}=(1-\alpha)^t \hat{\epsilon}_1+\alpha \sum_{\tau=0}^{t-1}(1-\alpha)^\tau \epsilon_{t+1-\tau}+(1-\alpha) \sum_{\tau=0}^{t-1}(1-\alpha)^\tau Z\left(\vec{w}_{t-\tau}, \vec{w}_{t+1-\tau}\right)+\alpha \sum_{\tau=0}^{t-1}(1-\alpha)^\tau Z\left(\vec{x}_{t+1}, \vec{w}_{t+1}\right)
$$

次に、
$$
\begin{aligned}
& \alpha\left\|Z\left(\vec{x}_t, \vec{w}_t\right)\right\| \leq \rho \frac{(1-\alpha)^2 \eta^2}{\alpha} \leq \rho \frac{(1-\alpha) \eta^2}{\alpha} \\
& (1-\alpha)\left\|Z\left(\vec{w}_t, \vec{w}_{t+1}\right)\right\| \leq(1-\alpha) \rho \eta^2 \leq \rho \frac{(1-\alpha) \eta^2}{\alpha}
\end{aligned}
$$

定理1と同じように期待値をとります。
$$
\begin{aligned}
\mathbb{E}\left[\left\|\hat{\epsilon}_{t+1}\right\|\right] & \leq(1-\alpha)^t \sigma+\alpha \sqrt{\sum_{\tau=0}^{t-1}(1-\alpha)^{2 \tau} \sigma^2}+2 \frac{\eta^2 \rho}{\alpha} \sum_{\tau=0}^{t-1}(1-\alpha)^{\tau+1} \\
& \leq(1-\alpha)^t \sigma+\frac{\alpha \sigma}{\sqrt{1-(1-\alpha)^2}}+2 \frac{\eta^2 \rho(1-\alpha)}{\alpha^2} \\
& \leq(1-\alpha)^t \sigma+\sqrt{\alpha} \sigma+2 \frac{\eta^2 \rho(1-\alpha)}{\alpha^2}
\end{aligned}
$$

次に、$t$ について和を取ります。

$$
\sum_{t=1}^T \mathbb{E}\left[\left\|\hat{\epsilon}_t\right\|\right] \leq \frac{\sigma\left(1-\alpha^T\right)}{\alpha}+T \sqrt{\alpha} \sigma+2 \frac{T \eta^2 \rho(1-\alpha)}{\alpha^2}
$$

次に、補題2を適用しましょう。

$$
\sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq \frac{3 F\left(\vec{w}_1\right)}{\eta}+\frac{3 L T \eta}{2}+\frac{8 \sigma\left(1-\alpha^T\right)}{\alpha}+8 T \sqrt{\alpha} \sigma+16 \frac{T \eta^2 \rho(1-\alpha)}{\alpha^2}
$$

 $\eta=\min \left(\frac{R^{5 / 7}}{T^{5 / 7} \rho^{1 / 7} \sigma^{4 / 7}}, \sqrt{\frac{R}{T L}}\right)$ とします。

$$
\frac{3 F\left(\vec{w}_1\right)}{\eta}+\frac{3 L T \eta}{2} \leq 5 \sqrt{R L T}+3 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7}
$$

$\alpha=\min \left(\frac{R^{4 / 7} \rho^{2 / 7}}{T^{4 / 7} \sigma^{6 / 7}}, 1\right)$ とします。$T \sqrt{\alpha} \sigma \leq R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7}$ であるので、

$$
\sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq 5 \sqrt{R L T}+11 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7}+\frac{8 \sigma\left(1-\alpha^T\right)}{\alpha}+16 \frac{T \eta^2 \rho(1-\alpha)}{\alpha^2} \quad (4)
$$

$\alpha=1$ について、

$$
\sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq 5 \sqrt{R L T}+11 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7} \quad (5)
$$

$\alpha=\frac{R^{4 / 7} \rho^{2 / 7}}{T^{4 / 7} \sigma^{6 / 7}}$ とします。 $\eta=\frac{R^{5 / 7}}{T^{5 / 7} \rho^{1 / 7} \sigma^{4 / 7}}$ という事実を用いて、

$$
\begin{aligned}
& \sum_{t=1}^T \mathbb{E}\left[\left\|\nabla F\left(\vec{w}_t\right)\right\|\right] \leq 5 \sqrt{R L T}+11 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7}+\frac{8 \sigma}{\alpha}+16 \frac{T \eta^2 \rho}{\alpha^2} \\
& 5 \sqrt{R L T}+27 R^{2 / 7} \rho^{1 / 7} \sigma^{4 / 7} T^{5 / 7}+\frac{8 \sigma^{13 / 7} T^{4 / 7}}{R^{4 / 7} \rho^{2 / 7}} \quad (6)
\end{aligned}
$$

あとはいろいろすると定理3の右辺になります。